In [1]:
import yaml
import spotipy
import urllib as url
import random
import pickle
from spotipy.oauth2 import SpotifyOAuth

In [2]:
creds = yaml.load(open('/Users/daverappeport/.spotify/creds.yml','r'), Loader=yaml.FullLoader)
redirect_url="http://localhost:9001/callback"
scope = ["user-library-read", "playlist-modify-private","playlist-modify-public"]

In [3]:
auth_manager=SpotifyOAuth(client_id=creds.get('client_id'), client_secret=creds.get('client_secret'), redirect_uri=redirect_url, scope=scope)
sp=spotipy.Spotify(auth_manager=auth_manager)

In [4]:
results = sp.current_user_saved_tracks()

In [5]:
results.keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [6]:
results.get('items')[0].get('track').keys()

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track_number', 'type', 'uri'])

In [7]:
user=sp.current_user()

In [8]:
for plist in sp.user_playlists(user=user['id']).get('items'):
    print(plist.get('name'))

My Peloton Music by RaveDapeport
Golden Hour
Rapp Haus
Ghost Town
Asian Dinner
Vibes
Cruise
The Long and Winding Road
good shit
Workout
Starred
Gimme Some More
Liked from Radio
GDTG


In [9]:
running_seeds_url="https://open.spotify.com/playlist/5NrLk0qUyICYEG8cZWK7SL"
res_playlist_id=url.parse.urlparse(running_seeds_url).path.split('/')[2]

In [10]:
running_recs_url="https://open.spotify.com/playlist/3Elnf5eDmqXA9dC52Dr9UJ"
rec_playlist_id=url.parse.urlparse(running_recs_url).path.split('/')[2]

In [11]:
seed_tracks = sp.playlist_items(playlist_id=res_playlist_id)

In [12]:
seed_track_ids = [track.get('track').get('id') for track in seed_tracks.get('items')]

In [13]:
# seed_track_ids

In [19]:
res = []
for trackid in seed_track_ids: 
    res = res + sp.recommendations(seed_tracks=[trackid]).get('tracks')
len(res)

240

In [24]:
rec_track_ids = set([track.get('id') for track in res])

In [27]:
for track in res:
    print(track.get('name'), ',', track.get('artists')[0].get('name'))

We Belong , RAC
Still Young , The Cat Empire
Heartbeat , Claptone
6's to 9's (feat. Rationale) , Big Wild
The Real Deal (radio Edit) , GoldFish
Woke up in Bangkok (feat. Martin Gallop) , Deepend
Swear Like a Sailor , Tep No
Wonderful , Chris Malinchak
Southern Man , Akshin Alizadeh
Baby , Bakermat
Let Go , RAC
Walking in the Sun , PANG!
Everything Is Changing (feat. Julia Church) , GoldFish
Montrose Ave , Y.V.E. 48
Home , Moontricks
Birds , Chef'Special
Glory Box , The Avener
Wait and See (Deep Chills Remix) , Deep Chills
Show Me , Big Wild
So Good to Me - Extended Mix , Chris Malinchak
With Me , Lane 8
ONLY , ZHU
Sleepless In NYC , Autograf
Impact (feat. Robyn & Channel Tres) , SG Lewis
Dímelo , Snakehips
Numb , ZHU
Lost You (feat. Twin Shadow & D'Angelo Lacy) , Zeds Dead
Hello Clouds - Original Mix , Justin Martin
Sugar - Original Mix , Moon Boots
Blast Off (feat. Pharrell Williams) , Gesaffelstein
Molecules , Disclosure
The Longest Road - deadmau5 Remix Edit , Morgan Page
Mi Rumba ,

In [29]:
### if you need to clear the playlist
sp.playlist_remove_all_occurrences_of_items(
    playlist_id=rec_playlist_id, 
    items=[track.get('track').get('id') for track in sp.playlist_items(playlist_id=rec_playlist_id).get('items')]
)

{'snapshot_id': 'MTEsNThjOGEyMjc0ODg3YzVlMjcyYzU0MjE2ODAxYmJhMzU1NzQ4NGU0YQ=='}

In [30]:
# add new items to running recs playlist
for item in rec_track_ids:
    sp.playlist_add_items(playlist_id=rec_playlist_id, items=[item])

In [ ]:
# audio features
random.choice(list(sp.audio_features(tracks=rec_track_ids)))

In [ ]:
#### let's get my 300 most recent saved tracks
saved_tracks = []

for n in range(0,6):
    tracks = sp.current_user_saved_tracks(limit=50, offset=n*50)
    saved_tracks = saved_tracks + [t.get('track') for t in tracks.get('items')]

print(len(saved_tracks))

In [ ]:
# res.get('items')[0].get('track')
new_music = []

for item in saved_tracks:
    
    seed_id = item.get('id')
    recs = sp.recommendations(seed_tracks=[seed_id])
    
    for track in recs.get('tracks'):
        new_music.append(track)

In [ ]:
# and now we have 6k new tracks
print(len(new_music))

# random.choice(new_music)

In [ ]:
new_music_dict = {}

for m in new_music:
    new_music_dict[m.get('id')] = m
    
# get audio features
for n in range(0, int(len(new_music)/50)):
    features = sp.audio_features(tracks=[t.get('id') for t in new_music[n*50:(n+1)*50]])
    
    for f in features:
        new_music_dict[f.get('id')]['audio_features'] = f

In [ ]:
# k = random.choice(list(new_music_dict.keys()))
# new_music_dict[k]

In [ ]:
saved_music_dict = {} 

for m in saved_tracks:
    saved_music_dict[m.get('id')] = m
    
# get audio features
for n in range(0, int(len(saved_tracks)/50)):
    features = sp.audio_features(tracks=[t.get('id') for t in saved_tracks[n*50:(n+1)*50]])
    
    for f in features:
        saved_music_dict[f.get('id')]['audio_features'] = f

In [ ]:
# saved_music_dict.get(random.choice(list(saved_music_dict.keys())))

In [ ]:
# save spotify data to pickle file    
with open('new_music_dict.pickle', 'wb') as handle:
    pickle.dump(new_music_dict, handle)

In [ ]:
# save spotify data to pickle file    
with open('saved_music_dict.pickle', 'wb') as handle:
    pickle.dump(saved_music_dict, handle)

In [ ]:
total_ms = 0
for k in new_music_dict.keys():
    total_ms+=new_music_dict[k]['audio_features']['duration_ms']

total_hours = total_ms/1e3/60/60
print(f'We have {total_hours} of music in our data')

In [ ]:
len(new_music_dict.keys())